## Section 03: Pre-Flop Equity
**Abstract**: This notebook estimates the equity of different hands in Texas Hold'em poker at the Pre-Flop stage of the game using a Monte Carlo simulation approach. It processes a previously generated dataset of poker hands and engineers features that can be used to group similar hands. Then it calculates the frequency that certain types of hands will win, a useful approximation of equity. These equity approximations are saved as a dataframe for future use.

In [10]:
# Imports
import numpy as np
import pandas as pd
from deuces import Card

**Preflop Equity Calculation**: We estimate preflop equity by grouping hands based on their rank combinations and suitedness, and then calculating the frequency of winning for each group for various amounts of players. This is done by considering all possible opponent combinations and counting the number of times a hand wins at showdown based on showdown order and number of players.

In [11]:
"""
strip longform data into a dataframe that represents only hole cards and showdown order
"""

# load long-form dataframe
hands_long = pd.read_pickle('../data/hands_long.pkl')

# remove all unnecessary columns, we are only interested in the player's hole cards and showdown order
preflop_hands = hands_long[[
    'hole_', 'showdown_order_',
]].copy()
# preview data
preflop_hands.head()

,hole_,showdown_order_
0,"[533255, 67144223]",2
1,"[4212241, 164099]",4
2,"[268454953, 8394515]",5
3,"[8406803, 8398611]",4
4,"[16787479, 279045]",8


In [12]:
"""
Turn hole cards into a a column for rank set (unique integers) and suited (boolean)
"""
preflop_hands['rank_set'] = preflop_hands['hole_'].apply(
    lambda cards: frozenset([Card.get_rank_int(card) for card in cards])
)
preflop_hands['suited'] = preflop_hands['hole_'].apply(
    lambda cards: Card.get_suit_int(cards[0]) == Card.get_suit_int(cards[1])
)
preflop_hands = preflop_hands.drop(columns=['hole_'])
# preview data
preflop_hands.head()

,showdown_order_,rank_set,suited
0,2,"(10, 3)",False
1,4,"(1, 6)",False
2,5,"(12, 7)",False
3,4,(7),False
4,8,"(8, 2)",False


In [13]:
"""
create a dataframe that represents all the possible preflop conditions to account for, like number of players, rank combinations, and suitedness
"""
rows = []
for players in range(2,10): # number of players from 2 to 9 (includes self)
    for r1 in range(13): # rank 0 to 12 (2 to A)
        for r2 in range(13): # rank 0 to 12 (2 to A)
            rows.append({
                'players': players,
                'rank_set': frozenset([r1, r2]), # use frozenset to ignore order and easy comparisons
                'suited': r1 < r2, # suited if r1 < r2 (e.g., (A, K) is suited, (K, A) is not)
            })
preflop_conditions = pd.DataFrame(rows)
# preview data
preflop_conditions.head()

,players,rank_set,suited
0,2,(0),False
1,2,"(0, 1)",True
2,2,"(0, 2)",True
3,2,"(0, 3)",True
4,2,"(0, 4)",True


In [14]:
"""
merge the two dataframes to get all combinations preflop features and number of players
"""
preflop_hands = preflop_hands.merge(
    preflop_conditions,
    on=['rank_set', 'suited'],
)
# we should have 900k * 8 = 7.2 million rows now
preflop_hands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200000 entries, 0 to 7199999
Data columns (total 4 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   showdown_order_  int64 
 1   rank_set         object
 2   suited           bool  
 3   players          int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 171.7+ MB


To calculate the number of times a hand would win at showdown given its showdown order $O \in [1,9]$ and number of players $P \in [2,9]$, as well as the total number of hands to consider for $P$ players, we can use combinatorial mathematics.

When trying to determine how many hands we need to consider for $P$ players, we can think of it as choosing $P-1$ opponents from the 8 total opponents (since one player is the target). Thus, the total number of hands to consider for $P$ players is given by:
$$
\text{total hands} = \binom{8}{P-1}
$$
When trying to determine how many times a hand would win at showdown given its showdown order $O$ and number of players $P$, there are $9-O$ players with worse showdown order (9 total players - O players with less than or equal showdown order). We need to choose $P-1$ opponents from these $9-O$ players. Thus, the number of times a hand would win at showdown is given by:
$$
\text{wins} = \binom{9 - O}{P-1}
$$
These methods are defined in the functions `total_hands` and `wins` which belong to the `src.equity_features` module.

In [15]:
"""
add columns that represent the total number of hands to consider for P players, and the number of times the hand won (showdown_order_ is minimum compared to a combination of P-1 other players).
"""

from src.equity_features import total_hands, wins

preflop_hands['total_hands'] = preflop_hands['players'].apply(total_hands)

preflop_hands['wins'] = preflop_hands[['showdown_order_','players']].apply(
    lambda row: wins(row['showdown_order_'], row['players']),
    axis=1
)

# preview data
preflop_hands.head()

,showdown_order_,rank_set,suited,players,total_hands,wins
0,2,"(10, 3)",False,2,8,7
1,2,"(10, 3)",False,3,28,21
2,2,"(10, 3)",False,4,56,35
3,2,"(10, 3)",False,5,70,35
4,2,"(10, 3)",False,6,56,21


In [16]:
"""
group preflop features and sum the equity features (total hands and wins)
"""

preflop_equity = preflop_hands.groupby(['rank_set', 'suited', 'players']).sum()
preflop_equity = preflop_equity.drop(columns=['showdown_order_'])
preflop_equity.reset_index(inplace=True)
preflop_equity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rank_set     1352 non-null   object
 1   suited       1352 non-null   bool  
 2   players      1352 non-null   int64 
 3   total_hands  1352 non-null   int64 
 4   wins         1352 non-null   int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 43.7+ KB


In [ ]:
"""
convert rankset to separate columns to allow csv export
"""
def split_rankset(row):
    if row['suited']:
        return pd.Series(sorted(list(row['rank_set']), reverse=True))
    elif len(row['rank_set']) == 2:
        return pd.Series(sorted(list(row['rank_set'])))
    else:
        return pd.Series([list(row['rank_set'])[0], list(row['rank_set'])[0]])

preflop_hands[['r1', 'r2']] = preflop_hands.apply(split_rankset, axis=1)
preflop_equity = preflop_hands.drop(columns=['rank_set'])
# preview data
preflop_equity.head()

In [9]:
preflop_equity.to_csv('../data/preflop_equity.csv')

**Conclusion**: The estimated Pre-Flop equities can be a valuable tool for players to make informed decisions during the Pre-Flop stage of Texas Holdem poker, as well as a look-up table for poker AI models. The same methodology can be extended to estimate equity at later stages of the game, such as the Flop, Turn, and River, by incorporating community cards into the simulation.